## Load dependency files

In [ ]:
from js import fetch

for url in ['https://raw.githubusercontent.com/ray306/Running_LiB/main/files/structures.py', 
            'https://raw.githubusercontent.com/ray306/Running_LiB/main/files/LiB.py', 
            'https://raw.githubusercontent.com/ray306/Running_LiB/main/files/br-text.txt', 
            'https://raw.githubusercontent.com/ray306/Running_LiB/main/files/br-phono.txt']:
    filename = url.split('/')[-1]
    with open(filename, 'w') as f:
        res = await fetch(url)
        text = await res.text()
        f.write(text)

## Demo of Segmentation 

In [ ]:
import LiB
model = LiB.model

'Define the train/test corpus'
corpus = model.create_corpus('br-text.txt')
'New and init a memory'
m = model(corpus, lexicon_in=0.25, lexicon_out=0.0001, update_rate=0.2, life=3)

'RUN!!!'
m.logs['note'] = 0
for epoch_id in range(0,201):
    m.run(epoch_id, article_length=200, test_interval=100);
            
print(f"Time cost for training: {sum(m.logs['time_cost'])//60} min {sum(m.logs['time_cost'])%60:.1f}s\n")

print('Raw/LiB segmentation:')
m.show_result(corpus[:10])